# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.10 which is incompatible.


In [2]:
!pip install tensorboardX==1.4

    100% |████████████████████████████████| 71kB 3.0MB/s ta 0:00:011


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [3]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
#env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [7]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
rewards = env_info.rewards
dones = env_info.local_done
print(len(rewards))
print(len(dones))
print(states.shape)
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
20
20
(20, 33)
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   7.12805176e+00  -1.00000000e+00
  -3.63192368e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
   3.92812490e-02]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.029999999329447746


When finished, you can close the environment.

In [6]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tensorboardX import SummaryWriter

import os
import sys
import copy
import time
import math
import random
import numpy as np
import collections
from collections import deque

from unityagents import UnityEnvironment

In [4]:
GAMMA = 0.99
BATCH_SIZE = 128
LEARNING_RATE = 1e-4
REPLAY_SIZE = 100000
REPLAY_INITIAL = 10000
REWARD_STEPS = 1

TEST_ITERS = 1000

Vmax = 10
Vmin = -10
N_ATOMS = 51
DELTA_Z = (Vmax - Vmin) / (N_ATOMS - 1)

In [5]:
Experience = collections.namedtuple('ExperienceFirstLast', ('state', 'action', 'reward', 'next_state', 'done'))

In [6]:
class ExperienceSource:
    def __init__(self, brain_name, env, agent, tracker, steps_count=2):
        self.brain_name = brain_name
        self.env_info = env.reset(train_mode=True)[self.brain_name]
        self.agent = agent
        self.tracker = tracker
        self.steps_count = steps_count
        self.total_rewards = np.zeros(num_agents)
        self.next_states = None
        self.episode = 1
        self.episode_time = 0.0
        self.ts = time.time()
        
    def __iter__(self):
        time_s = 0.0
        while True:
            if self.next_states is None:
                states = self.env_info.vector_observations
            else:
                states = self.next_states
            actions = self.agent(states)
            self.env_info = env.step(actions)[self.brain_name]
            rewards = self.env_info.rewards
            self.total_rewards += rewards
            dones = self.env_info.local_done
            self.next_states = self.env_info.vector_observations
            if np.any(dones):
                exp = Experience(state=states, action=actions, reward=rewards, 
                                 next_state=self.next_states, done=dones)
                self.episode_time = time.time() - self.ts
                self.ts = time.time()
                solved = self.tracker.reward(self.total_rewards, self.episode, self.episode_time)
                if solved:
                    break
                self._reset()
                yield (exp)
            else:
                yield(Experience(state=states, action=actions, reward=rewards, 
                                 next_state=self.next_states, done=dones))
            
    def _reset(self):
        self.env_info = env.reset(train_mode=True)[self.brain_name]
        self.total_rewards = np.zeros(num_agents)
        self.next_states = None
        self.episode += 1

In [7]:
class ReplayBuffer:
    def __init__(self, act_size, buffer_size, batch_size, exp_source, device="cpu"):
        assert isinstance(exp_source, ExperienceSource)
        self.action_size = act_size
        self.capacity = buffer_size
        self.memory = deque(maxlen=self.capacity)
        self.batch_size = batch_size
        self.exp_source_iter = iter(exp_source)
        self.device = device
        self.idx = 0
        
    def __len__(self):
        return len(self.memory)
        
    def _add(self, sample):
        if len(self.memory) < self.capacity:
            self.memory.append(sample)
        else:
            self.memory[self.idx] = sample
        self.idx = (self.idx + 1) % self.capacity

    def populate(self, samples):
        for _ in range(samples):
            entry = next(self.exp_source_iter)
            state = entry.state
            action = entry.action
            reward = entry.reward
            next_state = entry.next_state
            done = entry.done
            for i in range(num_agents):
                exp = Experience(state=state[i,:], action=action[i,:], reward=reward[i], next_state=next_state[i,:],
                                 done=done[i])
                self._add(exp)

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(self.device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).to(self.device)
        
        return (states, actions, rewards, next_states, dones)

In [8]:
def float32_preprocessor(states):
    np_states = np.array(states, dtype=np.float32)
    return torch.tensor(np_states)

In [9]:
class Actor(nn.Module):
    def __init__(self, obs_size, act_size, h1=400, h2=300):
        super(Actor, self).__init__()
        
        self.net = nn.Sequential(nn.Linear(obs_size, h1),
                                nn.ReLU(),
                                nn.Linear(h1, h2),
                                nn.ReLU(),
                                nn.Linear(h2, act_size),
                                nn.Tanh(),
                                )
                
    def forward(self, x):
        return self.net(x)

In [10]:
class Critic(nn.Module):
    def __init__(self, obs_size, act_size, h1=400, h2=300):
        super(Critic, self).__init__()
        
        self.obs_net = nn.Sequential(nn.Linear(obs_size, h1),
                                    nn.ReLU(),
                                    )
        
        self.out_net = nn.Sequential(nn.Linear(h1 + act_size, h2),
                                    nn.ReLU(),
                                    nn.Linear(h2, 1),
                                    )
        
    def forward(self, x, a):
        obs = self.obs_net(x)
        return self.out_net(torch.cat([obs, a], dim=1))

In [11]:
class TargetNet:
    def __init__(self, model):
        self.model = model
        self.target_model = copy.deepcopy(model)

    def alpha_sync(self, alpha):
        assert isinstance(alpha, float)
        assert 0.0 < alpha <= 1.0
        state = self.model.state_dict()
        tgt_state = self.target_model.state_dict()
        for k, v in state.items():
            tgt_state[k] = tgt_state[k] * alpha + (1 - alpha) * v
        self.target_model.load_state_dict(tgt_state)

In [12]:
class Agent:
    def __init__(self, net, device="cpu", epsilon=0.3):
        self.net = net
        self.device = device
        self.epsilon = epsilon

    def __call__(self, states):
        states_v = float32_preprocessor(states).to(self.device)
        mu_v = self.net(states_v)
        actions = mu_v.data.cpu().numpy()
        actions += self.epsilon * np.random.normal(size=actions.shape)
        actions = np.clip(actions, -1, 1)
        return actions

In [13]:
class RewardTracker:
    def __init__(self, writer, num_agents, min_ts_diff=1.0):
        """
        Constructs RewardTracker
        :param writer: writer to use for writing stats
        :param min_ts_diff: minimal time difference to track speed
        """
        self.writer = writer
        self.min_ts_diff = min_ts_diff
        self.mean_reward = []
        self.rewards_window = deque(maxlen=100)

    def __enter__(self):
        self.ts = time.time()
        return self

    def __exit__(self, *args):
        self.writer.close()

    def reward(self, rewards, frame, time_s):
        self.mean_reward.append(np.mean(rewards))
        self.rewards_window.append(np.mean(rewards))
        print("Done %d episodes, mean reward %.3f, speed %.2f s/e" % (frame, np.mean(rewards), time_s))
        sys.stdout.flush()
        self.writer.add_scalar("episode_length", time_s, frame)
        self.writer.add_scalar("reward_100", np.mean(self.rewards_window), frame)
        self.writer.add_scalar("mean_reward", np.mean(rewards), frame)
        self.writer.add_scalar("min_reward", np.min(rewards), frame)
        self.writer.add_scalar("max_reward", np.max(rewards), frame)
        return self.mean_reward if np.mean(self.rewards_window) >= 30 else None
        #if np.mean(rewards_window) >= 30:
            #break

In [14]:
if __name__ == "__main__":
    save_path = os.path.join("saves", "d4pg-")# "ppo-" + args.name)
    os.makedirs(save_path, exist_ok=True)
    
    env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')    
    
    brain_name = env.brain_names[0]
    brain = env.brains[brain_name]
    
    # reset the environment
    env_info = env.reset(train_mode=True)[brain_name]
    # number of agents
    num_agents = len(env_info.agents)
    print('Number of agents:', num_agents)
    # size of each action
    action_size = brain.vector_action_space_size
    print('Size of each action:', action_size)
    # examine the state space 
    states = env_info.vector_observations
    state_size = states.shape[1]
    print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   
    
    net_act = Actor(state_size, action_size).to(device)
    net_critic = Critic(state_size, action_size).to(device)
    print(net_act)
    print(net_critic)
    tgt_act = TargetNet(net_act)
    tgt_critic = TargetNet(net_critic)
    
    writer = SummaryWriter()
    with RewardTracker(writer, num_agents=num_agents) as tracker:
        agent = Agent(net_act, device=device)
        exp_source = ExperienceSource(brain_name, env, agent, tracker=tracker, steps_count=1)
        buffer = ReplayBuffer(action_size, REPLAY_SIZE, BATCH_SIZE, exp_source, device=device)
    
        opt_act = optim.Adam(net_act.parameters(), lr=LEARNING_RATE)
        opt_critic = optim.Adam(net_critic.parameters(), lr=LEARNING_RATE)
    
        frame_idx = 0
        while True:
            frame_idx += 1
            buffer.populate(1)

            if len(buffer) < BATCH_SIZE:
                continue

            states_v, actions_v, rewards_v, next_states_v, dones_mask = buffer.sample()
            """print("states_v {}".format(states_v.shape))
            print("actions_v {}".format(actions_v.shape))"""
            #print("rewards_v {}".format(rewards_v.shape))
            """print("next_states_v {}".format(next_states_v.shape))
            print("dones_mask {}".format(dones_mask.shape))"""

            # train critic
            opt_critic.zero_grad()
            q_v = net_critic(states_v, actions_v)
            #print("critic_distr {}".format(q_v.shape))
            next_act_v = tgt_act.target_model(next_states_v)
            #print(tgt_critic.target_model(next_states_v, next_act_v).shape)
            q_next_v = tgt_critic.target_model(next_states_v, next_act_v)
            #print("next_distr {}".format(q_next_v.shape))
            q_next_v[dones_mask] = 0.0
            #print(proj_distr_v.shape)
            q_ref_v = rewards_v + q_next_v * GAMMA
            #print(q_ref_v.shape)
            critic_loss_v = F.mse_loss(q_v, q_ref_v.detach())
            critic_loss_v.backward()
            opt_critic.step()
            writer.add_scalar("loss_critic", critic_loss_v, frame_idx)

            # train actor
            opt_act.zero_grad()
            cur_actions_v = net_act(states_v)
            actor_loss_v = -net_critic(states_v, cur_actions_v)
            actor_loss_v = actor_loss_v.mean()
            actor_loss_v.backward()
            opt_act.step()
            writer.add_scalar("loss_actor", actor_loss_v, frame_idx)

            tgt_act.alpha_sync(alpha=1 - 1e-3)
            tgt_critic.alpha_sync(alpha=1 - 1e-3)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
Actor(
  (net): Sequential(
    (0): Linear(in_features=33, out_features=400, bias=True)
    (1): ReLU()
    (2): Linear(in_features=400, out_features=300, bias=True)
    (3): ReLU()
    (4): Linear(in_features=300, out_features=4, bias=True)
    (5): Tanh()
  )
)
Critic(
  (obs_net): Sequential(
    (0): Linear(in_features=33, out_features=400, bias=True)
    (1): ReLU()
  )
  (out_net): Sequential(
    (0): Linear(in_features=404, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=1, bias=True)
  )
)
Done 1 episodes, mean reward 0.161, speed 14.69 s/e
Done 2 episodes, mean reward 0.841, speed 14.93 s/e
Done 3 episodes, mean reward 1.262, speed 15.02 s/e
Done 4 episodes, mean reward 1.089, speed 14.99 s/e
Done 5 episodes, mean reward 0.870, speed 15.20 s/e
Done 6 episodes, mean reward 1.080, speed 15.18 s/e
Done 7 episodes, mean reward 1.045, s

StopIteration: 

In [ ]:
env.close()